In [1]:
import tensorflow as tf

/home/jeet/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#define weight and bias functions
def weights(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

def bias(shape):
    initial1 = tf.constant(0.1, shape=shape)
    return tf.Variable(initial1)

In [3]:
def conv2d(X,W):
    layer = tf.nn.conv2d(X,W, strides=[1,1,1,1], padding = 'SAME')
    return layer

def maxpool(X):
    layer = tf.nn.max_pool(X ,ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')
    return layer

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
sess = tf.InteractiveSession()

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [7]:
#1st convolutional layer
w1 = weights([5,5,1,32])
b1 = bias([32])
print(tf.size(w1))

Tensor("Size:0", shape=(), dtype=int32)


In [8]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [9]:
conv1 = tf.nn.relu(conv2d(x_image, w1)+b1)
pool1 = maxpool(conv1)
print(tf.size(pool1))

Tensor("Size_1:0", shape=(), dtype=int32)


In [10]:
#2nd Conc layer
w2 = weights([5,5,32,64])
b2 = bias([64])

In [11]:
conv2 = tf.nn.relu(conv2d(pool1, w2)+b2)
pool2 = maxpool(conv2)

In [12]:
w_fc1 = weights([7*7*64, 1024])
b_fc1 = bias([1024])

In [15]:
pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
fc1 = tf.nn.relu(tf.matmul(pool2_flat, w_fc1)+b_fc1)

In [19]:
#Dropout Layer
keep_prob = tf.placeholder(tf.float32)
fc1_drop = tf.nn.dropout(fc1, keep_prob)

In [22]:
w_fc2 = weights([1024, 10])
b_fc2 = bias([10])

y_conv = tf.matmul(fc1_drop, w_fc2) + b_fc2

In [24]:
#Preparing the graph

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [25]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
step 100, training accuracy 0.74
step 200, training accuracy 0.78
step 300, training accuracy 0.92
step 400, training accuracy 0.9
step 500, training accuracy 0.96
step 600, training accuracy 0.94
step 700, training accuracy 0.96
step 800, training accuracy 0.98
step 900, training accuracy 1
step 1000, training accuracy 1
step 1100, training accuracy 0.94
step 1200, training accuracy 1
step 1300, training accuracy 0.96
step 1400, training accuracy 0.98
step 1500, training accuracy 1
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 0.96
step 1900, training accuracy 0.96
test accuracy 0.9776
